In [1]:
#r "./bin/Debug/net5.0/PreStormCore.dll"
#r "./bin/Debug/net5.0/Demo.dll"
using System.Diagnostics;
using System.Xml.Linq;
using PreStormCore;

var sfcrimes = new Sfcrimes.Service();
var usa = new Usa.Service();

var crimes = sfcrimes.CrimeLocations
    // .Download(null, "outSR=4326", true, 8)
    .Download(null, "outSR=4326")
    .ToArray();

var cities = usa.Cities.Download(null, null, true, 4).ToArray();
var highways = usa.Highways.Download().ToArray();
var states = usa.States.Download().ToArray();

static void Show(this IEnumerable<dynamic> features) {
    features.OfType<Feature>().Take(1000).ToKml(placemarkElements: x => new[]{ new XElement("Style",
        new XElement("IconStyle",
            new XElement("color", "ffffffff"),
            new XElement("Icon",
                new XElement("href", "http://maps.google.com/mapfiles/kml/shapes/square.png"))),
        new XElement("PolyStyle",
            new XElement("color", "b3ffffff"))
        )})
        .Save("doc.kml");
    Process.Start(new ProcessStartInfo("doc.kml"){ UseShellExecute = true });
}


In [1]:
#r "nuget: XPlot.Plotly, 3.0.1"
XPlot.Plotly.Chart.Column(crimes
    .GroupBy(x => x.Pddistrict)
    .OrderByDescending(x => x.Count())
    .Select(x => Tuple.Create(x.Key, x.Count()))).Show()
     

Installed package XPlot.Plotly version 3.0.1

In [1]:
#r "nuget: XPlot.Plotly, 3.0.1"
XPlot.Plotly.Chart.Pie(crimes
    .GroupBy(x => x.Category)
    .OrderByDescending(x => x.Count())
    .Select(x => Tuple.Create(x.Key, x.Count()))).Show()


In [1]:
cities.Where(x => x.St == "OH").Show()


In [1]:
highways.Show()


In [1]:
states.Where(x => x.StateAbbr == "AK").Show()


In [1]:
using RedlandsEmergencyVehicles;

var redlands = new RedlandsEmergencyVehicles.Service();
var ambulance = new Ambulance {
    Unitname = "Dummy",
    Status = UnitStatus.Active,
    Speed = 40,
    Lastreport = DateTime.UtcNow,    
    Geometry = (-80, 45)
};

var result = await redlands.Ambulances.InsertAsync(ambulance);
result.InsertedFeatures


index,Unitname,Status,Type,Callnumber,Speed,Lastreport,Geometry,OID,IsDataBound,IsDirty
0,Dummy,Active,<null>,<null>,40,2021-02-17 00:39:39Z,PreStormCore.Point,394634,True,False


In [1]:
var redlands = new RedlandsEmergencyVehicles.Service();
var ambulances = redlands.Ambulances.Download("Unitname='Dummy'").ToArray();

var result = await redlands.Ambulances.DeleteAsync(ambulances);
result.Raw.deleteResults


index,objectId,success
0,394634,True


In [1]:
using Wildfire;
using static System.Linq.Enumerable;

var wildfire = new Wildfire.Service();
int width = 30, height = 20, iterations = 60;
var description = "COVID-19";

static int[][] Tick(this int[][] g) {
    int[][] f1(Func<int, int, int> f) => g.Select((r, y) => r.Select((_, x) => f(x, y)).ToArray()).ToArray();
    int f2(int[][] g, int x, int y) => g.ElementAtOrDefault(y)?.ElementAtOrDefault(x) ?? 0;
    var d = new[] { -1, 0, 1 };
    var sum = f1((x, y) => (from dx in d from dy in d where dx != 0 || dy != 0 select f2(g, x + dx, y + dy)).Sum());
    return f1((x, y) => {
        var count = f2(sum, x, y);
        return count == 3 || (count == 2 && f2(g, x, y) == 1) ? 1 : 0;
    });
}

static Polygon[] ToPolygons(this int[][] g) => g
    .SelectMany((r, y) => r.Select((value, x) => new { x, y, value }))
    .Where(p => p.value == 1)
    .Select(p => new Polygon((p.x, p.y), (p.x, p.y + 1), (p.x + 1, p.y + 1), (p.x + 1, p.y)){ spatialReference = 4326 })
    .ToArray();

var random = new Random(DateTime.Now.Millisecond);
var cache = new Dictionary<string, object>();
var grid = Range(0, height).Select(y => Range(0, width).Select(x => random.NextDouble() < 0.5 ? 0 : 1).ToArray()).ToArray();
WildfireResponsePolygon[] fires;

for(var i = 0; i < iterations; i++) {
    fires = grid.ToPolygons()
        .Select(p => new WildfireResponsePolygon { Geometry = p, Symbolid = 1, Description = description })
        .ToArray();
    fires.Show();
    System.Threading.Thread.Sleep(500);
    var next = grid.Tick();
    var key = string.Join("", next.SelectMany(p => p));
    if(cache.ContainsKey(key))
        break;
    cache.Add(key, null);
    grid = next;
}

var deletes = wildfire.WildfireResponsePolygons.Download($"description='{description}'").ToArray();
await wildfire.WildfireResponsePolygons.DeleteAsync(deletes);
await wildfire.WildfireResponsePolygons.InsertAsync(fires);
